# Preparação dos Dados

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("telecomx_limpo.csv")

In [5]:
print("Dimensão inicial:", df.shape)
df.head()

Dimensão inicial: (7267, 22)


,customerID,Evasao,Genero,Idoso,customer.Partner,customer.Dependents,Meses_como_cliente,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,Faturamento_Mensal,Faturamento_Total,Contas_Diarias
0,0002-ORFBO,NaN,Female,0,NaN,NaN,9,NaN,NaN,DSL,...,No,Yes,Yes,No,One year,NaN,Mailed check,65.6,593.3,2.186667
1,0003-MKNFE,NaN,Male,0,NaN,NaN,9,NaN,NaN,DSL,...,No,No,No,Yes,Month-to-month,NaN,Mailed check,59.9,542.4,1.996667
2,0004-TLHLJ,NaN,Male,0,NaN,NaN,4,NaN,NaN,Fiber optic,...,Yes,No,No,No,Month-to-month,NaN,Electronic check,73.9,280.85,2.463333
3,0011-IGKFF,NaN,Male,1,NaN,NaN,13,NaN,NaN,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,NaN,Electronic check,98.0,1237.85,3.266667
4,0013-EXCHZ,NaN,Female,1,NaN,NaN,3,NaN,NaN,Fiber optic,...,No,Yes,Yes,No,Month-to-month,NaN,Mailed check,83.9,267.4,2.796667


In [6]:
colunas_remover = ["customerID"]
df = df.drop(columns=colunas_remover, errors="ignore")

In [7]:
df_encoded = pd.get_dummies(df, drop_first=True)

In [8]:
taxa_evasao = df["Evasao"].value_counts(normalize=True)
print("Proporção das classes (%):\n", taxa_evasao * 100)

Proporção das classes (%):
 Series([], Name: proportion, dtype: float64)


In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
X = df_encoded.drop(columns=["Evasao"])
y = df_encoded["Evasao"]

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Preparação finalizada! Dados prontos para modelagem.")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Preparação finalizada! Dados prontos para modelagem.


# Correlação e Seleção de Variáveis

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(12, 8))
corr = df_encoded.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação - df_encoded')
plt.show()

In [ ]:
print("Correlação das variáveis com Evasao:")
print(corr["Evasao"].sort_values(ascending=False))

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='Evasao', y='TempoContrato', data=df)
plt.title('Tempo de Contrato vs Evasão')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='Evasao', y='TotalGasto', data=df)
plt.title('Total Gasto vs Evasão')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='TotalGasto', y='TempoContrato', hue='Evasao', data=df)
plt.title('Dispersão: Total Gasto × Tempo de Contrato com Evasão')
plt.show()

# Modelagem Preditiva

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
X = df_encoded.drop(columns=["Evasao"])
y = df_encoded["Evasao"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_logreg = LogisticRegression(random_state=42, max_iter=1000)
model_logreg.fit(X_train_scaled, y_train)

In [ ]:
def avalia_modelo(model, X_teste, y_teste, nome_modelo):
    y_pred = model.predict(X_teste)
    acc = accuracy_score(y_teste, y_pred)
    prec = precision_score(y_teste, y_pred)
    rec = recall_score(y_teste, y_pred)
    f1 = f1_score(y_teste, y_pred)
    cm = confusion_matrix(y_teste, y_pred)

    print(f"--- {nome_modelo} ---")
    print(f"Acurácia: {acc:.4f}")
    print(f"Precisão: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Matriz de Confusão:")
    print(cm)

    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusão - {nome_modelo}')
    plt.ylabel('Valor Real')
    plt.xlabel('Predito')
    plt.show()

    print("\nRelatório Completo:")
    print(classification_report(y_teste, y_pred))
    print("\n")


In [ ]:
avalia_modelo(model_logreg, X_test_scaled, y_test, "Regressão Logística")
avalia_modelo(model_rf, X_test, y_test, "Random Forest")

In [ ]:
print("Análise crítica:\n")
print("""
- A Regressão Logística exigiu normalização porque ela usa distância e magnitudes nas variáveis para calcular probabilidades e coeficientes. Sem a normalização, variáveis com escalas maiores dominariam o modelo.

- O Random Forest é baseado em árvores de decisão, que dividem os dados com base em regras de valores, portanto não depende da escala dos dados e não precisa de normalização.

- Comparando os resultados, observe qual modelo teve melhor equilíbrio entre acurácia, precisão, recall e F1-score.

- Verifique a matriz de confusão para entender se algum modelo está tendo muitos falsos positivos ou falsos negativos.

- Se um modelo tem desempenho muito bom no treino mas ruim no teste, pode estar com overfitting. Se desempenho ruim em ambos, pode estar underfitting.

- Ajustes podem incluir mudar hiperparâmetros, fazer seleção de variáveis, ou aumentar a base de dados.
""")

# Interpretação e Conclusões

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
coeficientes = pd.Series(model_logreg.coef_[0], index=X.columns)
coeficientes_sorted = coeficientes.sort_values(key=abs, ascending=False)

print("Coeficientes da Regressão Logística (ordenados pelo valor absoluto):")
print(coeficientes_sorted.head(10))

In [ ]:
plt.figure(figsize=(10,6))
coeficientes_sorted.head(10).plot(kind='barh', color='steelblue')
plt.title('Top 10 Variáveis mais Impactantes - Regressão Logística')
plt.xlabel('Coeficiente')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
importances = pd.Series(model_rf.feature_importances_, index=X.columns)
importances_sorted = importances.sort_values(ascending=False)

print("Importância das variáveis pelo Random Forest:")
print(importances_sorted.head(10))

plt.figure(figsize=(10,6))
importances_sorted.head(10).plot(kind='barh', color='darkorange')
plt.title('Top 10 Variáveis mais Importantes - Random Forest')
plt.xlabel('Importância')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
relatorio = """
Com base na análise dos dois modelos, observamos as variáveis que mais influenciam a evasão de clientes.

- Variáveis com coeficientes maiores na Regressão Logística indicam um aumento ou redução na probabilidade de evasão conforme seus valores aumentam.

- As variáveis mais importantes no Random Forest mostram onde o modelo encontra maior ganho ao fazer as divisões para separar clientes que evadem dos que permanecem.

Principais fatores identificados (exemplo, substitua pelos seus resultados reais):

- Tempo de contrato curto está relacionado a maior evasão.
- Clientes com altos custos totais tendem a evadir mais.
- Serviços específicos (ex: Internet, Streaming) podem impactar positivamente ou negativamente.
- Planos com múltiplas assinaturas adicionais mostram relação com evasão.

**Estratégias de retenção sugeridas:**

- Oferecer planos com contratos mínimos mais longos ou descontos para fidelização.
- Monitorar clientes com altos custos e oferecer pacotes personalizados.
- Avaliar os serviços que mais impactam a satisfação e promover melhorias.
- Campanhas direcionadas para grupos de risco identificados pelas variáveis relevantes.

Essa análise orienta ações estratégicas para reduzir a evasão, aumentando a retenção e o valor do cliente ao longo do tempo.
"""

print(relatorio)